In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

What I would like to know about this set of data?
1. The distribuition of gender, age, level of education, marital status, income Category and type of card of existing costumer and Attrited Customer (definir uma persona) - DONE!
2. Algoritmo de previsão de churn
3. Things that influence most to make people leave the credit card

In [ ]:
BankData = pd.read_csv('../input/credit-card-customers/BankChurners.csv')
BankData.columns

'1. The distribuition of gender, age, level of education, marital status, income Category and type of card of existing costumer and Attrited Customer (definir uma persona)'

In [ ]:
#Separation the data of Existing Clients and Attrited Clients
Exist_Clients = BankData.loc[BankData['Attrition_Flag']=='Existing Customer']
Attrited_Clients = BankData.loc[BankData['Attrition_Flag']=='Attrited Customer']


In [ ]:
Exist_Clients['Faixa Etária'] = np.where(Exist_Clients['Customer_Age']<=35,'26-35',np.where(Exist_Clients['Customer_Age']<=45,'36-45',np.where(Exist_Clients['Customer_Age']<= 55,'46-55',np.where(Exist_Clients['Customer_Age']>55,'56+','Outros'))))
Attrited_Clients['Faixa Etária'] = np.where(Attrited_Clients['Customer_Age']<=35,'26-35',np.where(Attrited_Clients['Customer_Age']<=45,'36-45',np.where(Attrited_Clients['Customer_Age']<= 55,'46-55',np.where(Attrited_Clients['Customer_Age']>55,'56+','Outros'))))


In [ ]:
#Creating the distribution to each variable we want to analyse
Gender_Exist_Clients = Exist_Clients.groupby('Gender').count()
Gender_Attrited_Clients = Attrited_Clients.groupby('Gender').count()

Education_Exist_Clients = Exist_Clients.groupby('Education_Level').count()
Education_Attrited_Clients = Attrited_Clients.groupby('Education_Level').count()

Marital_Exist_Clients = Exist_Clients.groupby('Marital_Status').count()
Marital_Attrited_Clients = Attrited_Clients.groupby('Marital_Status').count()

IncCategory_Exist_Clients = Exist_Clients.groupby('Income_Category').count()
IncCategory_Attrited_Clients = Attrited_Clients.groupby('Income_Category').count()
IncCategory_Exist_Clients['RKG'] = [0,3,2,1,4,5]
IncCategory_Exist_Clients.sort_values('RKG', ascending = True, inplace = True)

TypeCard_Exist_Clients = Exist_Clients.groupby('Card_Category').count()
TypeCard_Attrited_Clients = Attrited_Clients.groupby('Card_Category').count()

Age_Exist_Clients = Exist_Clients.groupby('Faixa Etária').count()
Age_Attrited_Clients = Attrited_Clients.groupby('Faixa Etária').count()

In [ ]:
def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width()/2., 1*height,
                '%d' % int(height),
                ha='center', va='bottom')
        
def oneorzero(i):
    if i == 'Existing Customer':
        return 1
    else:
        return 0

In [ ]:
#Finishing to preparing the data and ploting the graphics

#PLotting the gender data
fig = plt.figure(figsize=(21,21))
Labels = ['Existing Clients','Attrited Clients']
Genderplot = pd.DataFrame()
Genderplot['Existing Clients'] = Gender_Exist_Clients['CLIENTNUM']
Genderplot['Attrited Clients'] = Gender_Attrited_Clients['CLIENTNUM']
Genderplot['Gender2'] = ['Female','Male']
Genderplot.set_index('Gender2', inplace = True)


ax1 = plt.subplot2grid((3,3),(0,0), rowspan = 1, colspan = 1)
width = 0.35
x_axis = np.arange(len(Labels))
ract1 = ax1.bar(x_axis-width/2,Genderplot.loc['Male'], width = 0.35,color="#6494f5")
ract2 = ax1.bar(x_axis+width/2,Genderplot.loc['Female'], width = 0.35, color='#f77ed7' )
plt.xticks(x_axis, Labels)
plt.title('Gender Distributions for each type of client')
plt.legend(['Male','Female'])

autolabel(ract1)
autolabel(ract2)

#Plotting the Educational data
Educationplot = pd.DataFrame()
Educationplot['Existing Clients'] = Education_Exist_Clients['CLIENTNUM']
Educationplot['Attrited Clients'] = Education_Attrited_Clients['CLIENTNUM']


ax2 = plt.subplot2grid((3,3),(0,1), rowspan = 1, colspan = 1)
width = 0.35
x_axis2 = np.arange(len(['College','Doctorate','Graduate','High School','Post Graduate','Uneducated','Unknown']))
ract_3 = ax2.bar(x_axis2-width/2,Educationplot['Existing Clients'], width = 0.35, color='#bdf77e' )
ract_4 = ax2.bar(x_axis2+width/2,Educationplot['Attrited Clients'], width = 0.35, color='#f77e7e' )


plt.xticks(x_axis2,['College','Doctorate','Graduate','High School','Post Graduate','Uneducated','Unknown'], rotation = 15 )
plt.title('Educational Level for each type of client')
plt.legend(['Existing Clients','Attrited Clients'])

autolabel(ract_3)
autolabel(ract_4)

#Plotting the Age data
Ageplot = pd.DataFrame()
Ageplot['Existing Clients'] = Age_Exist_Clients['CLIENTNUM']
Ageplot['Attrited Clients'] = Age_Attrited_Clients['CLIENTNUM']
                                                               
ax3 = plt.subplot2grid((3,3),(0,2), rowspan = 1, colspan = 1)
ract_5 = plt.barh(Ageplot.index,Ageplot['Existing Clients'],color='#bdf77e')
ract_6 = plt.barh(Ageplot.index,-Ageplot['Attrited Clients'],color='#f77e7e')

plt.title("Age's Distribution for each type of clients")
plt.legend(['Existing Clients','Attrited Clients'])

#Plotting the Income Category data
Incplot = pd.DataFrame()
Incplot['Existing Clients'] = IncCategory_Exist_Clients['CLIENTNUM']
Incplot['Attrited Clients'] = IncCategory_Attrited_Clients['CLIENTNUM']

ax4 = plt.subplot2grid((3,3),(1,0), rowspan = 1, colspan = 1)
ract_7 = plt.barh(Incplot.index,Incplot['Existing Clients'],color='#bdf77e')
ract_8 = plt.barh(Incplot.index,-Incplot['Attrited Clients'],color='#f77e7e')

plt.title("Income Category distribution for each type of clients")
plt.legend(['Existing Clients','Attrited Clients'])

#Plotting the Marital Status data

Maritalplot = pd.DataFrame()
Maritalplot['Existing Clients'] = Marital_Exist_Clients['CLIENTNUM']
Maritalplot['Attrited Clients'] = Marital_Attrited_Clients['CLIENTNUM']

ax5 = plt.subplot2grid((3,3),(1,1), rowspan = 1, colspan = 1)
x_axis5 = np.arange(len(['Divorced','Married','Single','Unknown']))
ract_9 = ax5.bar(x_axis5-width/2,Maritalplot['Existing Clients'], width = 0.35, color='#bdf77e' )
ract_10 = ax5.bar(x_axis5+width/2,Maritalplot['Attrited Clients'], width = 0.35, color='#f77e7e' )

plt.xticks(x_axis5,['Divorced','Married','Single','Unknown'])
plt.title('Marital Status for each type of client')
plt.legend(['Existing Clients','Attrited Clients'])

autolabel(ract_9)
autolabel(ract_10)

#Plotting the Type of Card data
Typeplot = pd.DataFrame()
Typeplot['Existing Clients'] = TypeCard_Exist_Clients['CLIENTNUM']
Typeplot['Attrited Clients'] = TypeCard_Attrited_Clients['CLIENTNUM']

ax6 = plt.subplot2grid((3,3),(1,2), rowspan = 1, colspan = 1)
x_axis6 = np.arange(len(['Blue', 'Gold', 'Platinum','Silver']))
ract_11 = ax6.bar(x_axis6-width/2,Typeplot['Existing Clients'], width = 0.35, color='#bdf77e' )
ract_12 = ax6.bar(x_axis6+width/2,Typeplot['Attrited Clients'], width = 0.35, color='#f77e7e' )

plt.xticks(x_axis6,['Blue', 'Gold', 'Platinum','Silver'])
plt.title('Type of Card for each type of client')
plt.legend(['Existing Clients','Attrited Clients'])

autolabel(ract_11)
autolabel(ract_12)

In [ ]:
label_encoder = LabelEncoder()
onehot_encoder = skl.preprocessing.OneHotEncoder(sparse=False)



In [ ]:
#2. Algoritmo de previsão de churn
#Using Random Forests as a model.
Y_target = pd.DataFrame(BankData['Attrition_Flag'])
X_features = BankData.drop(['Attrition_Flag','CLIENTNUM'], axis = 1)

In [ ]:
X_features_numerical = onehot_encoder.fit_transform(X_features)
X_features_numerical.shape

X_features_categorical = pd.DataFrame()
X_features_categorical['Gender'] = X_features['Gender']
X_features_categorical['Education_Level'] = X_features['Education_Level']
X_features_categorical['Marital_Status'] = X_features['Marital_Status']
X_features_categorical['Income_Category'] = X_features['Income_Category']
X_features_categorical['Card_Category'] = X_features['Card_Category']
X_features.drop(['Gender','Education_Level','Marital_Status','Income_Category','Card_Category'], axis = 1, inplace = True)

X_Num_features =pd.DataFrame()
X_features_categorical['Gender'] = label_encoder.fit_transform(X_features_categorical['Gender'])
X_features_categorical['Education_Level'] = label_encoder.fit_transform(X_features_categorical['Education_Level']) 
X_features_categorical['Marital_Status'] = label_encoder.fit_transform(X_features_categorical['Marital_Status'])
X_features_categorical['Income_Category'] = label_encoder.fit_transform(X_features_categorical['Income_Category'])
X_features_categorical['Card_Category'] = label_encoder.fit_transform(X_features_categorical['Card_Category'])
X_features_categorical

In [ ]:
Y_target['Classe'] = Y_target['Attrition_Flag'].map(oneorzero)
Y_target.drop('Attrition_Flag', axis = 1, inplace = True)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X_features_numerical,Y_target,test_size=0.3)

In [ ]:
Scores = []
Max_depth = 1

In [ ]:
len(Scores)

In [ ]:
while(len(Scores) < 300):
    model = tree.DecisionTreeClassifier(max_depth = Max_depth)
    model = model.fit(X_train,Y_train)
    Scores.append(model.score(X_test, Y_test))
    Max_depth = Max_depth + 50

In [ ]:
plt.plot(Scores)